In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
    
%matplotlib inline

# Cleaning player bio info & pickling for later
* Deleting bad top records from Scrapy
* Scrapy script got lots of unicode/formatting characters, cleaning those up
    * Height
    * Throws With
    * Weight
    * Home City
* Home state must be 2 letters

In [209]:
# import csv
df_bio = pd.read_csv('player_f.csv')

In [210]:
# cleanup weird recs
df_bio = df_bio.iloc[2:].copy()

In [211]:
# cleanup weight
df_bio.iloc[672,3] = None
df_bio['weight'] = df_bio.apply(lambda x: x['weight'][:3] 
                                if x['weight'] is not None 
                                else x['weight'], axis = 1)

In [212]:
# cleanup height
df_bio.height = df_bio.apply(lambda x: None if x['height'] == '\n\t\t' else x['height'], axis=1)

df_bio.height = df_bio.apply(lambda x: int(x['height'][:1]) * 12 + int(x['height'][2:len(x['height'])]) 
                             if x['height'] is not None
                             else x['height'],
                            axis = 1)

In [213]:
# cleanup throws
throws_bad = [',\xa0','\n  ','\xa0(196cm,\xa099kg) ' ]
df_bio.throws = df_bio.apply(lambda x: None 
                             if x['throws'] in throws_bad 
                             else x['throws'], axis = 1)

def throws_clean(x):
    if x == '\n\t\tRight\n\t\n':
        return 'Right'
    elif x == '\n\t\tLeft\n\t\n':
        return 'Left'
    else:
        return x

df_bio.throws = df_bio.apply(lambda x: throws_clean(x['throws']), axis =1 )

In [214]:
# clean home state
df_bio.home_state = df_bio.apply(lambda x: None 
                                 if len(x['home_state']) != 2 
                                 else x['home_state'], axis=1)

In [215]:
# clean city - unicode = :-(
def clean_city(x):
    import string
    printable = set(string.printable)
    a =[s for s in x if s in printable]
    b = ''.join(a)
    return b

df_bio.home_city = df_bio.apply(lambda x: clean_city(x['home_city']), axis = 1)
df_bio.home_city = df_bio.apply(lambda x: x['home_city'].strip(), axis = 1)
df_bio.home_city = df_bio.apply(lambda x: x['home_city'][:len(x['home_city'])-1] 
                                if x['home_city'][-1:] == ',' 
                                else x['home_city'],
                                axis = 1)
df_bio.home_city = df_bio.apply(lambda x: x['home_city'][2:] 
                                if x['home_city'][:2] == 'in' 
                                else None,
                                axis = 1)


In [228]:
# pickle
with open('player_bio.pkl', 'wb') as picklefile:
    pickle.dump(df_bio, picklefile)